In [109]:
import numpy as np
import re

In [116]:
class BluePrint:
    def __init__(self, ore_robot_cost_ore, clay_robot_cost_ore, obsidian_robot_cost_ore, obsidian_robot_cost_clay, geode_robot_cost_ore, geode_robot_cost_obsidian):
        self.ore_robot_cost_ore = ore_robot_cost_ore
        self.clay_robot_cost_ore = clay_robot_cost_ore
        self.obsidian_robot_cost_ore = obsidian_robot_cost_ore
        self.obsidian_robot_cost_clay = obsidian_robot_cost_clay
        self.geode_robot_cost_ore = geode_robot_cost_ore
        self.geode_robot_cost_obsidian = geode_robot_cost_obsidian
        
    def return_max_ore_cost(self):
        return max(self.ore_robot_cost_ore, self.clay_robot_cost_ore, self.obsidian_robot_cost_ore, self.geode_robot_cost_ore)
    def print_(self):
        print(f'Ore robot cost ore: {self.ore_robot_cost_ore}. Clay robot cost ore: {self.clay_robot_cost_ore}. Obsidian robot cost ore: {self.obsidian_robot_cost_ore}. Obsidian robot cost clay: {self.obsidian_robot_cost_clay}. Geode robot cost ore: {self.geode_robot_cost_ore}. Geode robot cost obsidian: {self.geode_robot_cost_obsidian}')
        
class Deposit:
    def __init__(self, ore, clay, obsidian, geode):
        self.ore = ore
        self.clay = clay
        self.obsidian = obsidian
        self.geode = geode
        
    def copy(self):
        return Deposit(self.ore, self.clay, self.obsidian, self.geode)
        
class Robots:
    def __init__(self, ore, clay, obsidian, geode):
        self.ore = ore
        self.clay = clay
        self.obsidian = obsidian
        self.geode = geode
        
    def copy(self):
        return Robots(self.ore, self.clay, self.obsidian, self.geode)

In [117]:
with open('19.txt') as f:
    lines = f.readlines()

blueprints = []

for line in lines:
    match = re.search(r"ore robot.* (\d+) ore.*clay robot.* (\d+) ore.*obsidian robot.* (\d+) ore.* and (\d+) clay.*geode robot.* (\d+) ore.* and (\d+) obsidian", line)
    ore_cost = match.group(1)
    clay_cost = match.group(2)
    obsidian_ore_cost = match.group(3)
    obsidian_clay_cost = match.group(4)
    geode_ore_cost = match.group(5)
    geode_obsidian_cost = match.group(6)
    blueprint = BluePrint(int(ore_cost), int(clay_cost), int(obsidian_ore_cost), int(obsidian_clay_cost), int(geode_ore_cost), int(geode_obsidian_cost))
    blueprints.append(blueprint)

In [118]:
blueprints[0].print_()

Ore robot cost ore: 4. Clay robot cost ore: 2. Obsidian robot cost ore: 3. Obsidian robot cost clay: 14. Geode robot cost ore: 2. Geode robot cost obsidian: 7


In [135]:
def mine(robot, deposit):
        deposit.ore += robot.ore
        deposit.clay += robot.clay
        deposit.obsidian += robot.obsidian
        deposit.geode += robot.geode
        return deposit
    
def build(robots, deposit, which_robot):
    if which_robot == "ore":
        robots.ore += 1
        deposit.ore -= blueprint.ore_robot_cost_ore
    elif which_robot == "clay":
        robots.clay += 1
        deposit.ore -= blueprint.clay_robot_cost_ore
    elif which_robot == "obsidian":
        robots.obsidian += 1
        deposit.ore -= blueprint.obsidian_robot_cost_ore
        deposit.clay -= blueprint.obsidian_robot_cost_clay
    elif which_robot == "geode":
        robots.geode += 1
        deposit.ore -= blueprint.geode_robot_cost_ore
        deposit.obsidian -= blueprint.geode_robot_cost_obsidian
    else:
        print("Error: unknown robot")
    return robots, deposit

def get_possible_options(blueprint, deposit, robots):
    possible_options = ['wait']
    for robot in ['ore', 'clay', 'obsidian', 'geode']:
        if robot == 'ore':
            if deposit.ore >= blueprint.ore_robot_cost_ore and not robots.ore >= blueprint.return_max_ore_cost():
                possible_options.append(robot)
        elif robot == 'clay':
            if deposit.ore >= blueprint.clay_robot_cost_ore and not robots.clay >= blueprint.obsidian_robot_cost_clay:
                possible_options.append(robot)
        elif robot == 'obsidian':
            if deposit.ore >= blueprint.obsidian_robot_cost_ore and deposit.clay >= blueprint.obsidian_robot_cost_clay and not robots.obsidian >= blueprint.geode_robot_cost_obsidian:
                possible_options.append(robot)
        elif robot == 'geode':
            if deposit.ore >= blueprint.geode_robot_cost_ore and deposit.obsidian >= blueprint.geode_robot_cost_obsidian:
                possible_options.append(robot)
        else:
            pass
    return possible_options

def optimize_build_pattern(robots, deposit, blueprint, possible_option, minute, max_minutes):
    if minute > max_minutes:
        return robots, deposit
    if possible_option in ['ore', 'clay', 'obsidian', 'geode']:
        robots, deposit = build(robots, deposit, possible_option)
    possible_options = get_possible_options(blueprint, deposit, robots)
    deposit = mine(robots, deposit)
    
    possible_deposits = []
    possible_robots = []
    for option in possible_options:
        robots_, deposit_ = optimize_build_pattern(robots.copy(), deposit.copy(), blueprint, option, minute+1, max_minutes)
        possible_robots.append(robots_)
        possible_deposits.append(deposit_)
        
    max_idx = np.argmax([deposit.geode for deposit in possible_deposits])
    return possible_robots[max_idx], possible_deposits[max_idx]

In [136]:
blueprints[0].print_()

Ore robot cost ore: 4. Clay robot cost ore: 2. Obsidian robot cost ore: 3. Obsidian robot cost clay: 14. Geode robot cost ore: 2. Geode robot cost obsidian: 7


In [154]:
MAX_MINUTES = 24
deposit = Deposit(0, 0, 0, 0)
robots = Robots(1, 0, 0, 0)

all_robots, all_deposits, possibilities = [], [], []
robots, deposit = optimize_build_pattern(robots, deposit, blueprints[0], 'wait', minute=1, max_minutes=MAX_MINUTES)

In [ ]:
robots.ore

2

In [ ]:
robots.clay

5

In [ ]:
deposit.clay

26

In [ ]:
deposit.geode

1